In [10]:
#this script is a 'one-time' script. It will simply pull the locations of snotel sites
#and then these should get written to a shapefile...
import geopandas as gpd
import pandas as pd
import numpy as np
from datetime import datetime, timedelta, date
import requests
import json
from geojson import Point, Feature, FeatureCollection, dump
from rasterstats import point_query
from shapely import geometry as sgeom
import ulmo
from collections import OrderedDict
from pathlib import Path

In [11]:
#########################################################################
# SNOTEL Functions
#########################################################################

# functions to get SNOTEL stations as geodataframe
def sites_asgdf(ulmo_getsites, stn_proj):
    """ Convert ulmo.cuahsi.wof.get_sites response into a point GeoDataframe
    """
    
    # Note: Found one SNOTEL site that was missing the location key
    sites_df = pd.DataFrame.from_records([
        OrderedDict(code=s['code'], 
        longitude=float(s['location']['longitude']), 
        latitude=float(s['location']['latitude']), 
        name=s['name'], 
        elevation_m=s['elevation_m'])
        for _,s in ulmo_getsites.items()
        if 'location' in s
    ])

    sites_gdf = gpd.GeoDataFrame(
        sites_df, 
        geometry=gpd.points_from_xy(sites_df['longitude'], sites_df['latitude']),
        crs=stn_proj
    )
    return sites_gdf

In [14]:
def get_snotel_stns():
    
    #path to CSO domains
    domains_resp = requests.get("https://raw.githubusercontent.com/snowmodel-tools/preprocess_python/master/CSO_domains.json")
    domains = domains_resp.json()
    
    #USA
    Bbox = {"latmax": 72, "latmin": 18, "lonmin": -172, "lonmax": -66}
    #Bbox = {"latmax": 44, "latmin": 43.5, "lonmin": -127, "lonmax": -122}
    stn_proj = "epsg:4326"

    # Convert the bounding box dictionary to a shapely Polygon geometry using sgeom.box
    box_sgeom = sgeom.box(Bbox['lonmin'], Bbox['latmin'], Bbox['lonmax'], Bbox['latmax'])
    box_gdf = gpd.GeoDataFrame(geometry=[box_sgeom], crs=stn_proj)
    
    # WaterML/WOF WSDL endpoint url 
    wsdlurl = "https://hydroportal.cuahsi.org/Snotel/cuahsi_1_1.asmx?WSDL"
    
    # get dictionary of snotel sites 
    sites = ulmo.cuahsi.wof.get_sites(wsdlurl,user_cache=True)

    #turn sites to geodataframe 
    snotel_gdf = sites_asgdf(sites,stn_proj)
    
    #clip snotel sites to domain bounding box
    gdf = gpd.sjoin(snotel_gdf, box_gdf, how="inner")
    gdf.drop(columns='index_right', inplace=True)
    gdf.reset_index(drop=True, inplace=True)

    return gdf

In [15]:
#get snotel stations
snotel_gdf = get_snotel_stns()
print('Retrieved station locations')
#print(snotel_gdf)
cols=['geometry']
snotel_gdf_subset=snotel_gdf[cols]
print(snotel_gdf_subset)

Retrieved station locations
                        geometry
0    POINT (-120.79192 41.23583)
1    POINT (-112.27118 37.52217)
2    POINT (-111.95902 45.59723)
3    POINT (-150.88966 61.74967)
4    POINT (-121.69847 47.77957)
..                           ...
925  POINT (-110.91773 33.81242)
926  POINT (-111.35685 39.13233)
927  POINT (-112.77495 37.74797)
928  POINT (-109.81775 43.93225)
929  POINT (-106.59535 40.79488)

[930 rows x 1 columns]
